In [7]:
import datasets
from transformers import AutoTokenizer
import torch

In [8]:
data = datasets.load_dataset("stanfordnlp/imdb")['test']
# Так как промпты составляются для генератора, то берем его токенайзер
tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")

In [9]:
tokenizer.pad_token_id = tokenizer.unk_token_id # Изначально в токенайзере нет pad токена. Можно поставить любое значение, так как все равно потом от него избавимся

In [10]:
def tokenize_sample(sample, prompt_tokens_count=7):
    tokenized_prompt = tokenizer(sample['text'], padding='max_length', truncation=True, return_tensors='pt')
    tokenized_prompt['input_ids'] = tokenized_prompt['input_ids'][:, :prompt_tokens_count]
    tokenized_prompt['attention_mask'] = tokenized_prompt['attention_mask'][:,:prompt_tokens_count]
    ones_tensor = torch.ones_like(tokenized_prompt['attention_mask'], dtype=torch.int64)
    assert torch.equal(tokenized_prompt['attention_mask'], ones_tensor) # проверка, что все промпты имеют равное количество токенов
    sample['prompt_input_ids'] = tokenized_prompt['input_ids']
    sample['prompt_attention_mask'] = tokenized_prompt['attention_mask']
    return sample

data = data.map(tokenize_sample, num_proc=16, batched=True, batch_size=64)
data.set_format('torch')

Map (num_proc=16):   0%|          | 0/25000 [00:00<?, ? examples/s]

In [11]:
data.save_to_disk('../prompts_dataset_tokenized_test')

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]